In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# print(device_name)
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

> # Include packages and libraries

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras_preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras import backend as K
from keras.layers.core import Activation, Flatten, Dropout, Dense

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split

> # About the data

## Read file data 

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
df.info()

The data contains 2 columns.  
The first column is the *images' names* named '**image**', another column is the '**labels**' of all the images in the dataset.  
There are 18632 images and 18632 labels so that there are no null data in this dataset.

> # Process the data

## Visualize the data

In [ ]:
print(df['labels'].value_counts().plot.bar())

In [ ]:
df['labels'].value_counts()

## **NOTE:**  
### As we can see here, there are 12 labels but some labels are the combine other labels.  
### So that, there actually are 5 diseases which are:  
* rust
* scab
* complex
* frog_eye_leaf_spot
* powdery_mildew

### And another label is:  
* healthy

### Because one image (leaf) can have multiple diseases so that this task is a multi-label classification problem!!!

In [ ]:
y_tr = df['labels'].values
y_tr

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(y_tr)

In [ ]:
image_labels[:5]

In [ ]:
n_classes = len(label_binarizer.classes_)

In [ ]:
n_classes

In [ ]:
# device_name = tf.test.gpu_device_name()
# print(device_name)
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
height = 128 # 256 64
width = 128 # 256 64
depth = 3

In [ ]:
# # Read images function
# default_image_size = tuple((height,width))
# def convert_image_to_array(image_dir):
#     try:
#         image = cv2.imread(image_dir)
#         if image is not None :
#             image = cv2.resize(image, default_image_size)   
#             return img_to_array(image)
#         else :
#             return np.array([])
#     except Exception as e:
#         print(f"Error : {e}")
#         return None

In [ ]:
# # Create a list contains all img_to_aray(images)
# images = []
# for filename in os.listdir('../input/plant-pathology-2021-fgvc8/train_images'):
#     image_directory = os.path.join('../input/plant-pathology-2021-fgvc8/train_images',filename)
#     images.append(convert_image_to_array(image_directory))

In [ ]:
# len(images)

In [ ]:
# images[0].shape, images[0]

In [ ]:
# np_images = np.array(images, dtype=np.float16) / 255.0

In [ ]:
# np_images.shape, np_images[0].shape, np_images[0]

In [ ]:
# save file images
# np.save('np_images.npy', np_images)

In [ ]:
data = np.load(r'../input/np-images/np_images.npy')

In [ ]:
type(image_labels), image_labels.shape, image_labels[0]

In [ ]:
type(image_labels[0]), type(data)

In [ ]:
data[0].shape

In [ ]:
data.shape, image_labels.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(data, image_labels, test_size=0.2, random_state = 42)

In [ ]:
# aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
#     height_shift_range=0.1, shear_range=0.2, 
#     zoom_range=0.2,horizontal_flip=True, 
#     fill_mode="nearest")
aug = ImageDataGenerator()

In [ ]:
# input_shape = (height,width,depth)
# model = Sequential()
# model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=input_shape))
# model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
# model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
# model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(n_classes, activation='softmax'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

# model = Sequential()
# inputShape = (height, width, depth)
# chanDim = -1
# # if K.image_data_format() == "channels_first":
# #     inputShape = (depth, height, width)
# #     chanDim = 1
    
# model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
# model.add(Activation("relu"))
# model.add(BatchNormalization())#axis=chanDim
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization())#axis=chanDim

# model.add(Conv2D(64, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization())#axis=chanDim
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization())#axis=chanDim

# model.add(Conv2D(128, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization())#axis=chanDim
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(1024))
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(256))
# model.add(Activation("relu"))
# model.add(Dense(n_classes))
# model.add(Activation("softmax"))

# model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

model=Sequential()
model.add(Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(256, 256, depth)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
        
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(12))
model.add(Activation("softmax"))

model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=12,average='macro')

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001, decay=0.01/30), loss='categorical_crossentropy',metrics=['accuracy',f1])

In [ ]:
earlyStopping=EarlyStopping(
    patience=5,
    monitor=f1,
    mode='max',
    restore_best_weights=True
)
lrSchedule = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.05, 
    patience=4, 
    verbose=1
)

In [ ]:
Batch_size = 32
history = model.fit(
    aug.flow(x_train, y_train, batch_size=Batch_size),
    validation_data=(x_val, y_val),
#     steps_per_epoch=len(x_train) // Batch_size,
    epochs=25
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
sample_sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sample_sub

In [ ]:
test_images = []
for filename in os.listdir('../input/plant-pathology-2021-fgvc8/test_images'):
    image_directory = os.path.join('../input/plant-pathology-2021-fgvc8/test_images',filename)
    images.append(convert_image_to_array(image_directory))

In [ ]:
np_test_images = np.array(test_images, dtype=np.float16) / 255.0

In [ ]:
for i in range(3):
    result = model.predict(np_test_images[i])
    itemindex = np.where(result==np.max(result))
    sample_sub['labels'][i] = label_binarizer.classes_[itemindex[1][0]]

In [ ]:
sample_sub.to_csv('submission.csv', index=False)